In [221]:
from pyspark.sql import SparkSession

# New API
spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.113:7077") \
        .appName("JunjieChuA3PartA")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.shuffle.service.enabled", True)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores",2)\
        .config("spark.driver.port",9998)\
        .config("spark.blockManager.port",10005)\
        .getOrCreate()

# Old API (RDD)
spark_context = spark_session.sparkContext
spark_context.setLogLevel("INFO")

In [222]:
lines1 = spark_context.textFile("hdfs://192.168.2.113:9000/europarl/europarl-v7.pl-en.en")
lines2 = spark_context.textFile("hdfs://192.168.2.113:9000/europarl/europarl-v7.pl-en.pl")
print(lines1.count())
print(lines2.count())
print(lines1.getNumPartitions())
print(lines2.getNumPartitions())

632565
632565
2
2


In [223]:
def preprocessmap(lines):
    lowlines=lines.map(lambda line: line.lower())
    words=lowlines.map(lambda word: word.split(' '))
    return(words)

def preprocessflatmap(lines):
    lowlines=lines.map(lambda line: line.lower())
    words=lowlines.flatMap(lambda word: word.split(' '))
    return(words)
    

In [224]:
wordsE = preprocessmap(lines1)
wordsP = preprocessmap(lines2)
print('En 10:')
print(wordsE.take(10))
print('Pl 10:')
print(wordsP.take(10))
print(f'lines number of en: {wordsE.count()}\n')
print(f'lines number of pl: {wordsP.count()}\n')

En 10:
[['action', 'taken', 'on', "parliament's", 'resolutions:', 'see', 'minutes'], ['documents', 'received:', 'see', 'minutes'], ['written', 'statements', '(rule', '116):', 'see', 'minutes'], ['texts', 'of', 'agreements', 'forwarded', 'by', 'the', 'council:', 'see', 'minutes'], ['membership', 'of', 'parliament:', 'see', 'minutes'], ['membership', 'of', 'committees', 'and', 'delegations:', 'see', 'minutes'], ['future', 'action', 'in', 'the', 'field', 'of', 'patents', '(motions', 'for', 'resolutions', 'tabled):', 'see', 'minutes'], ['agenda', 'for', 'next', 'sitting:', 'see', 'minutes'], ['closure', 'of', 'sitting'], ['(the', 'sitting', 'was', 'closed', 'at', '11.55', 'p.m.)']]
Pl 10:
[['działania', 'podjęte', 'w', 'wyniku', 'rezolucji', 'parlamentu:', 'patrz', 'protokól'], ['składanie', 'dokumentów:', 'patrz', 'protokół'], ['oświadczenia', 'pisemne', '(art.', '116', 'regulaminu):', 'patrz', 'protokół'], ['teksty', 'porozumień', 'przekazane', 'przez', 'radę:', 'patrz', 'protokół'], ['s

In [225]:
words1=preprocessflatmap(lines1)
words2=preprocessflatmap(lines2)

def countwords(words):
    countwords = words\
        .map(lambda key: (key,1))\
        .reduceByKey(lambda x,y:x+y)\
        .sortBy(lambda x:x[1],False)
    return (countwords)

countwords1=countwords(words1)
countwords2=countwords(words2)

print(countwords1.take(10))
print(countwords2.take(10))

[('the', 1157014), ('of', 558149), ('to', 505756), ('and', 451165), ('in', 370228), ('a', 255921), ('that', 253400), ('is', 247419), ('for', 186707), ('we', 172260)]
[('w', 488954), ('i', 342893), ('na', 220274), ('z', 189566), ('do', 163373), ('że', 156657), ('się', 155444), ('nie', 138417), ('jest', 118263), ('to', 97921)]


In [ ]:
def preprocessmapindex(lines):
    lowlines=lines.map(lambda line: line.lower())
    words=lowlines.map(lambda word: word.split(' ')).zipWithIndex()
    wordsnew = words.map(lambda word:(word[1],word[0]))    
    return(wordsnew)

wordsa = preprocessmapindex(lines1)
wordsb = preprocessmapindex(lines2)
pairs = wordsa\
    .join(wordsb)\
    .sortBy(lambda x:x[0],True)

pairs = pairs\
    .filter(lambda x: len(x[1][0])!=0)\
    .filter(lambda x: len(x[1][1])!=0)\
    .filter(lambda x: len(x[1][0])<=5)\
    .filter(lambda x: len(x[1][1])<=5)\
    .filter(lambda x: len(x[1][1])==len(x[1][0]))

print(f'lines number: {pairs.count()}\n')
print(pairs.take(15))

pairs = pairs\
    .flatMap(lambda pair:(list(zip(pair[1][0],pair[1][1]))))\
    .map(lambda p:(p,1))\
    .reduceByKey(lambda x,y:x+y)\
    .sortBy(lambda x:x[1],False)

print('\n')
print(pairs.take(15))






In [ ]:
# release the cores for another application!
spark_context.stop()